In [95]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
%autoreload 2

In [97]:
import pktypes
from pktypes import *
from pkhands import *

In [127]:
txt = lambda x : '<p class="player-text black">'+str(x)+'</p>'

In [128]:
txt('ddssass')

'<p class="player-text black">ddssass</p>'

In [98]:
allCards1 = [Card('J','s'), Card('4','s'), Card('10','s'), Card('8','h'), 
                    Card('2','d'), Card('K','s'), Card('2','s')]
allCards2 = [Card('J','s'), Card('A','d'), Card('10','s'), Card('8','h'), 
                    Card('2','d'), Card('K','s'), Card('Q','s')]
hand1 = getHand(allCards1)
hand2 = getHand(allCards2)

In [71]:
print(hand1)
print(hand2)

(True, [2, 4, 10, 11, 13], 513, 'Flush')
(True, [10, 11, 12, 13, 14], 414, 'Straight')


In [61]:
#EVAL HANDS
d.repopulate()
d.shuffle()
plrHnad = [c for c in d.deal(2)]
commCards = [c for c in d.deal(5)]
allCards = plrHnad + commCards
for c in allCards:
    print(c)

As
Qs
3h
2c
7s
5h
9h


In [66]:
getHand(allCards)

(True, [1, 2, 3, 4, 5], 405, 'Straight')

In [65]:
allCards = [Card('J','h'), Card('3','d'), Card('4','c'), Card('5','s'), Card('2','d'), Card('2','s'), Card('A','d')]

In [ ]:
values = [v.value for v in allCards]
suits = [v.suit for v in allCards]
ranks = [v.rank for v in allCards]
valcts = Counter(values).values()
suitcts = Counter(suits).values()

In [ ]:
print(Counter(suits))
print(Counter(values))

In [ ]:
print(values)
print(suits)
print(ranks)
print(valcts)
print(suitcts)

In [ ]:
allCards1 = [Card('J','s'), Card('4','s'), Card('10','s'), Card('8','h'), 
                    Card('2','d'), Card('K','s'), Card('2','s')]
        allCards2 = [Card('J','s'), Card('A','d'), Card('10','s'), Card('8','h'), 
                    Card('2','d'), Card('K','s'), Card('Q','s')]
        hand1 = getHand(allCards1)
        hand2 = getHand(allCards2)

In [ ]:
getHand(values, suits, ranks)

In [ ]:
values = [9,9,7,6,6,8,10]
suits = ['h','d','d','c','h','s','s']
hand = set(values)
print(hand)
print(Counter(suits))

In [ ]:
allCards = [Card('5','d'), Card('5','d'), Card('5','d'), Card('7','d'), Card('7','s'), Card('7','d'), Card('A','d')]

In [ ]:
vals = [v.value for v in allCards]
suits = [v.suit for v in allCards]
ranks = [v.rank for v in allCards]

In [ ]:
checkStraight(vals)

In [ ]:
checkFlush(vals, suits)

In [ ]:
fvals = checkFlush(vals, suits)

In [ ]:
checkStraightFlush(fvals[1])

In [ ]:
checkQuads(vals)

In [ ]:
checkTrips(vals)

In [ ]:
checkPairs(vals)

In [ ]:
for low in range(10,0,-1):
    print(low)
    straight = set(range(low, low+5))
    print(straight)

In [ ]:
r = ['8','4','10','5','6','A','K']

In [ ]:
sorted(r)

In [ ]:
checkStraight(r)

In [ ]:
joe = Player('sdff', 'Joe', bank=44, stack=333)
fred = Player('sdf2', 'Fred', bank=44, stack=333)
willis = Player('sdf3', 'willis', bank=44, stack=333)
sally = Player('sdf9', 'Sally', bank=44, stack=333)
lucy = Player('sdf8', 'Lucy', bank=44, stack=333)
patty = Player('sdf7', 'Patty', bank=44, stack=333)

In [ ]:
print(joe)

In [ ]:
t = Table(5)

In [ ]:
t.addPlayerToLobby(joe)
t.addPlayerToLobby(fred)
t.addPlayerToLobby(willis)
t.addPlayerToLobby(sally)
t.addPlayerToLobby(lucy)
t.addPlayerToLobby(patty)

In [ ]:
joe.atTable

In [ ]:
for player in t.playersInLobby:
    player.sitDown()

In [ ]:
t.dealCommCards(1)

In [ ]:
joe.leaveTable()

In [ ]:
fred.standUp()

In [ ]:
print(t)

In [ ]:
t.startGame()

In [ ]:
print(t.buttonSeatNum)

In [ ]:
t.seatedPlayers

In [ ]:
t.getNextSeatNum(3)

In [ ]:
t.removePlayerFromTable(willis)

In [ ]:
t.resetRound()

In [ ]:
t.moveBtton()

In [ ]:
t.clearPlayerHands()
t.deal()

In [ ]:
print(list(range(5)))

In [ ]:
#init

#create Table
#While Table Online
    #for player in Table.players where player.seated == True
    #create list of seated players
    #get ante
        #update player investment
        #update pot 
    #deal
    #betting round
        #
    #flop
    #betting round
    #turn
    #betting round
    #river
    
    
    
    
    # Update player status (seated or standing)
    # update if table is online or offline



In [ ]:
d.clear()
d.populate()
d.shuffle()

In [ ]:
d.deal(4)

In [ ]:
f=[3,4,8,6]

In [ ]:
f.pop()

In [ ]:
x, y = input("Enter action and amount: ").split() 
print("Number of boys: ", x) 
print("Number of girls: ", y) 
print()

In [ ]:
i=4
j=7
while i<7:
    while j<30:
        print(i,j)
        i+=1
        j+=1

In [ ]:
import csv

with open('./testsims/sim.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

print(data)

In [ ]:
c=tuple([5,6])

In [ ]:
a,b=c

In [ ]:
a

In [ ]:
x=[['Call 0'], ['Raise', ' 10'], ['Fold', ' 0'], ['Raise', ' 100'], ['Call', ' 0'], ['Call', ' 0'], ['Call', ' 0'], ['Call', ' 0'], ['Fold', ' 0']]

In [ ]:
x

In [ ]:
x.pop(0)

In [ ]:
x

In [ ]:
for x in enumerate(values):
    print(x)

In [ ]:
values

In [81]:
for i in range(9,-1,-1):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [120]:
a = ['ddd',4,'ff',6,7]

In [124]:
len(a)

5

In [122]:
if a[5]:
    print("hi")

IndexError: list index out of range

In [119]:
a

['ddd', 'GGG', 4, 'ff', 6, 7]

In [89]:
for x in range(len(a)-1,-1,-1):
    print(x)

2
1
0


In [102]:
for i in enumerate(reversed(a)):
    print(i[1])

5
4
3


In [105]:
reversed(a)[1]

TypeError: 'list_reverseiterator' object is not subscriptable

In [111]:
d = next(a[4])

TypeError: 'int' object is not an iterator

In [5]:
rooms = [
	{
		'tableID': "Poker table 1",
		'noOfPlayers': 0,
		'online': True,
		'players': [],
		'tablePot': 0,
		'amount': 0,
		'dealer': False,
		'actions': {
			'bet': False,
			'check': False,
		},
		'options': {
			'amount': 20000,
			'smallBlind': 5,
			'bigBlind': 10,
		},
	}]

In [6]:
rooms

[{'actions': {'bet': False, 'check': False},
  'amount': 0,
  'dealer': False,
  'noOfPlayers': 0,
  'online': True,
  'options': {'amount': 20000, 'bigBlind': 10, 'smallBlind': 5},
  'players': [],
  'tableID': 'Poker table 1',
  'tablePot': 0}]

In [7]:
a = [1,2,3,4,5]

In [8]:
a.remove(4)

In [13]:
a

[1, 2, 3, 5]

In [12]:
list(range(1,10+1)) 

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
a.pop()

5

In [16]:
room_dict = {}

In [19]:
room_dict['table1'] = {'actions': {'bet': False, 'check': False},
  'amount': 0,
  'dealer': False,
  'noOfPlayers': 0,
  'online': True,
  'options': {'amount': 20000, 'bigBlind': 10, 'smallBlind': 5},
  'players': [{'table': 1, 'seat': 2, 'name': "mate", 'amount': 2000, 'status': 'hold' },
              { 'table': 1, 'seat': 1, 'name': "pero", 'amount': 2320, 'status': 'hold' },],
  'tableID': 'table1',
  'tablePot': 0}

room_dict['table2'] = {'tableID': 'table2', 'online': True,}
room_dict['table3'] = {'tableID': 'table3', 'online': True,}

roomList = [room_dict[x] for x in room_dict.keys()]

In [ ]:
roomList

[{'actions': {'bet': False, 'check': False},
  'amount': 0,
  'dealer': False,
  'noOfPlayers': 0,
  'online': True,
  'options': {'amount': 20000, 'bigBlind': 10, 'smallBlind': 5},
  'players': [{'amount': 2000,
    'name': 'mate',
    'seat': 2,
    'status': 'hold',
    'table': 1},
   {'amount': 2320, 'name': 'pero', 'seat': 1, 'status': 'hold', 'table': 1}],
  'tableID': 'table1',
  'tablePot': 0},
 {'online': True, 'tableID': 'table3'},
 {'online': True, 'tableID': 'table2'}]